# Motion Correction using CaImAn (Multiple Videos)

This notebook performs piecewise rigid registration in order to correct motion in multiple videos. It is a modified version of the work published by the Flatiron Institute: https://github.com/flatironinstitute/CaImAn/blob/main/demos/notebooks/demo_motion_correction.ipynb.

Most of the libraries used can be easily installed through Anaconda or by using the "pip install" command. A tutorial on installing CaImAn can be found here: https://caiman.readthedocs.io/en/latest/Installation.html.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os.path
import logging
import os

# Set the directory to where you would like the final motion corrected videos to be
os.chdir('C:\\Users\\user1\\Documents\\Motion Corrected')

try:
    cv2.setNumThreads(0)
except:
    pass

try:
    if __IPYTHON__:
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.motion_correction import MotionCorrect, tile_and_correct, motion_correction_piecewise
from caiman.utils.utils import download_demo

In [ ]:
max_shifts = (6, 6)  # maximum allowed rigid shift in pixels (view the movie to get a sense of motion)
strides =  (48, 48)  # create a new patch every x pixels for pw-rigid correction
overlaps = (24, 24)  # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 3   # maximum deviation allowed for patch with respect to rigid shifts
pw_rigid = False  # flag for performing rigid or piecewise rigid motion correction
shifts_opencv = True  # flag for correcting motion using bicubic interpolation (otherwise FFT interpolation is used)
border_nan = 'copy'  # replicate values along the boundary (if True, fill in with NaN)

In [ ]:
# start the cluster (if a cluster already exists terminate it)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
# Enter filenames without extension (we assume .tif is the extension, your video should be converted to an appropriate type accepted by CaImAn)
vid_to_correct = ["video1","video2","video3","video4","video5","video6"]
for video in vid_to_correct:
    fnames = f"{video}.tif"
    filen = video
    fnames = [download_demo(fnames)]     # the file will be downloaded if it doesn't already exist
    m_orig = cm.load_movie_chain(fnames)
    
    # create a motion correction object
    mc = MotionCorrect(fnames, dview=dview, max_shifts=max_shifts,
                  strides=strides, overlaps=overlaps,
                  max_deviation_rigid=max_deviation_rigid, 
                  shifts_opencv=shifts_opencv, nonneg_movie=True,
                  border_nan=border_nan)

    # correct for rigid motion correction and save the file (in memory mapped form)
    mc.motion_correct(save_movie=True)
    # load motion corrected movie
    m_rig = cm.load(mc.mmap_file)
    bord_px_rig = np.ceil(np.max(mc.shifts_rig)).astype(int)

    # motion correct piecewise rigid
    mc.pw_rigid = True  # turn the flag to True for pw-rigid motion correction
    mc.template = mc.mmap_file  # use the template obtained before to save in computation (optional)

    mc.motion_correct(save_movie=True, template=mc.total_template_rig)
    m_els = cm.load(mc.fname_tot_els)

    # Save the video as a .tif file (with an appropriate filename)
    output_tif_filename = f"{filen}_corrected.tif"
    m_els.save(output_tif_filename)

In [ ]:
# You can choose to use a software package such as ImageJ/Fiji to visualise the corrected videos
# Alternatively, follow the methods to visualise the videos in the CaImAn documentation